<a href="https://colab.research.google.com/github/yucelasan/gz-dbt-repository/blob/main/3A_Superstore_M%C3%BC%C5%9Fteri_Segmentasyonu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## BigQuery ile datasetlerin colabe import edilmesi.

In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
from google.cloud import bigquery

client = bigquery.Client(project="superstore-484710")


In [3]:
import pandas as pd
query = """
SELECT * FROM `superstore-484710.dbt_superstore_mart.order_overview`
"""
orders = client.query(query).to_dataframe()
orders.head()


,order_id,order_date,branch_id,market_name,region,city,customer_town,market_location,user_id,name_surname,user_age,order_age,payment_type,product_count,total_quantity,total_revenue
0,1286099,2022-08-19,667-ZO2,3A-Zonguldak-ZONGULDAK MERKEZ,Karadeniz,Zonguldak,ZONGULDAK MERKEZ,ZONGULDAK MERKEZ,13943,Çağrı Kadir Bayram,27,26,Yemek Kartı,8,31,12045.97
1,305982,2023-02-08,759-TE2,3A-Tekirdağ-ÇORLU,Marmara,Tekirdağ,ÇORLU,ÇORLU,94417,Ali Yaman,53,53,Yemek Kartı,4,24,10792.26
2,1725940,2022-09-08,550-NE1,3A-Nevşehir-NEVŞEHİR MERKEZ,İç Anadolu,Nevşehir,ÜRGÜP,NEVŞEHİR MERKEZ,59102,Doğukan Mustafa Gündüz,48,47,Yemek Kartı,2,10,1837.96
3,4070917,2021-09-16,463-ŞA3,3A-Şanlıurfa-SİVEREK,Güneydoğu Anadolu,Şanlıurfa,SİVEREK,SİVEREK,63717,Gizem Esila Köse,43,41,Yemek Kartı,2,12,2713.20
4,1400442,2021-01-05,213-BI1,3A-Bitlis-BİTLİS MERKEZ,Doğu Anadolu,Bitlis,TATVAN,BİTLİS MERKEZ,94396,Nazan Güneş,28,26,Yemek Kartı,1,2,287.48


## RFM Skorlarının Hesaplanması

Kişilerin ilk siparişlerinin tutulması.

In [4]:
import pandas as pd


first_order = (
    orders.groupby("user_id")["order_date"]
    .min()
    .reset_index()
    .rename(columns={"order_date": "first_order_date"})
)

first_order.head()

,user_id,first_order_date
0,1,2021-01-18
1,2,2021-02-26
2,3,2021-01-10
3,4,2021-01-15
4,5,2021-01-12


İlk siparişlerin işaretlenmesinden sonraki siparişlerin 'Returning' olarak adlandırılması.

In [5]:
import pandas as pd

orders["order_date"] = pd.to_datetime(orders["order_date"], errors="coerce")

first_order = (
    orders.groupby("user_id")["order_date"]
    .min()
    .reset_index()
    .rename(columns={"order_date": "first_order_date"})
)

first_order.head()


,user_id,first_order_date
0,1,2021-01-18
1,2,2021-02-26
2,3,2021-01-10
3,4,2021-01-15
4,5,2021-01-12


In [6]:
orders2 = orders.merge(first_order, on="user_id", how="left")

orders2["customer_type"] = orders2.apply(
    lambda x: "New" if x["order_date"] == x["first_order_date"] else "Returning",
    axis=1
)

orders2["customer_type"].value_counts()

,count
customer_type,
Returning,10129825
New,105368


In [7]:
import pandas as pd

orders2["customer_type"] = "Returning"
orders2.loc[orders2["order_date"] == orders2["first_order_date"], "customer_type"] = "New"

orders2["customer_type"].value_counts()

,count
customer_type,
Returning,10129825
New,105368


In [8]:
new_rate = (orders2["customer_type"] == "New").mean()
print("New Customer Rate:", new_rate)

New Customer Rate: 0.010294676416946901


In [9]:
returning_rate = (orders2["customer_type"] == "Returning").mean()
print("Returning Rate:", returning_rate)

Returning Rate: 0.9897053235830531


Order Date in data type i datetime olarak ayarlandı. Sonrasında aylar bazında New ve Returning customerlar incelendi.

In [13]:
orders["order_date"] = pd.to_datetime(orders["order_date"], errors="coerce")
orders2["order_date"] = pd.to_datetime(orders2["order_date"], errors="coerce")
orders2["month"] = orders2["order_date"].dt.to_period("M").astype(str)

monthly_ct = (
    orders2.groupby(["month", "customer_type"])["order_id"]
    .nunique()
    .reset_index()
)

monthly_ct.head(10)

,month,customer_type,order_id
0,2021-01,New,101691
1,2021-01,Returning,231083
2,2021-02,New,3494
3,2021-02,Returning,296717
4,2021-03,New,177
5,2021-03,Returning,332208
6,2021-04,New,6
7,2021-04,Returning,321856
8,2021-05,Returning,332218
9,2021-06,Returning,321499


Elimizdeki datayı göz önünde bulundurarak ilk 3 ayı çıkartarak returning customer ları tespit etmenin doğru olacağına karar verildi.

In [14]:
monthly_ct_filtered = monthly_ct[
    ~monthly_ct["month"].isin(sorted(monthly_ct["month"].unique())[:3])
]
monthly_ct_filtered.head(10)

,month,customer_type,order_id
6,2021-04,New,6
7,2021-04,Returning,321856
8,2021-05,Returning,332218
9,2021-06,Returning,321499
10,2021-07,Returning,332025
11,2021-08,Returning,331874
12,2021-09,Returning,320835
13,2021-10,Returning,332093
14,2021-11,Returning,320924
15,2021-12,Returning,332074


RFM skorları hesaplandı.

In [15]:
orders2.head()

,order_id,order_date,branch_id,market_name,region,city,customer_town,market_location,user_id,name_surname,user_age,order_age,payment_type,product_count,total_quantity,total_revenue,first_order_date,customer_type,month
0,1286099,2022-08-19,667-ZO2,3A-Zonguldak-ZONGULDAK MERKEZ,Karadeniz,Zonguldak,ZONGULDAK MERKEZ,ZONGULDAK MERKEZ,13943,Çağrı Kadir Bayram,27,26,Yemek Kartı,8,31,12045.97,2021-01-09,Returning,2022-08
1,305982,2023-02-08,759-TE2,3A-Tekirdağ-ÇORLU,Marmara,Tekirdağ,ÇORLU,ÇORLU,94417,Ali Yaman,53,53,Yemek Kartı,4,24,10792.26,2021-01-07,Returning,2023-02
2,1725940,2022-09-08,550-NE1,3A-Nevşehir-NEVŞEHİR MERKEZ,İç Anadolu,Nevşehir,ÜRGÜP,NEVŞEHİR MERKEZ,59102,Doğukan Mustafa Gündüz,48,47,Yemek Kartı,2,10,1837.96,2021-02-05,Returning,2022-09
3,4070917,2021-09-16,463-ŞA3,3A-Şanlıurfa-SİVEREK,Güneydoğu Anadolu,Şanlıurfa,SİVEREK,SİVEREK,63717,Gizem Esila Köse,43,41,Yemek Kartı,2,12,2713.20,2021-01-05,Returning,2021-09
4,1400442,2021-01-05,213-BI1,3A-Bitlis-BİTLİS MERKEZ,Doğu Anadolu,Bitlis,TATVAN,BİTLİS MERKEZ,94396,Nazan Güneş,28,26,Yemek Kartı,1,2,287.48,2021-01-05,New,2021-01


In [19]:
snapshot_date = orders2["order_date"].max() + pd.Timedelta(days=1)

rfm = orders2.groupby("user_id").agg(
    Recency=("order_date", lambda x: (snapshot_date - x.max()).days),
    Frequency=("order_id", "nunique"),
    Monetary=("total_revenue", "sum")
).reset_index()
rfm["segment"] = "Regular"
rfm.head()


,user_id,Recency,Frequency,Monetary,segment
0,1,2,113,1132620.00,Regular
1,2,24,112,1121866.99,Regular
2,3,6,86,912249.56,Regular
3,4,6,98,917517.91,Regular
4,5,22,102,938302.75,Regular


Monetary için round fonksiyonu eklendi.

In [20]:
rfm["Monetary"] = rfm["Monetary"].round(2)
rfm.head()

,user_id,Recency,Frequency,Monetary,segment
0,1,2,113,1132620.00,Regular
1,2,24,112,1121866.99,Regular
2,3,6,86,912249.56,Regular
3,4,6,98,917517.91,Regular
4,5,22,102,938302.75,Regular


In [21]:
# Skorlar: 1 (düşük) - 5 (yüksek)
rfm["R_score"] = pd.qcut(rfm["Recency"], 5, labels=[5,4,3,2,1])   # düşük recency daha iyi
rfm["F_score"] = pd.qcut(rfm["Frequency"].rank(method="first"), 5, labels=[1,2,3,4,5])
rfm["M_score"] = pd.qcut(rfm["Monetary"].rank(method="first"), 5, labels=[1,2,3,4,5])

# string -> int
rfm["R_score"] = rfm["R_score"].astype(int)
rfm["F_score"] = rfm["F_score"].astype(int)
rfm["M_score"] = rfm["M_score"].astype(int)

# Toplam skor
rfm["RFM_score"] = rfm["R_score"] + rfm["F_score"] + rfm["M_score"]

rfm["segment"].value_counts()


,count
segment,
Regular,99996


In [22]:
import numpy as np
import pandas as pd

# 1) Monetary temizle (formatta sorun olmaması için dönüşümler yapıldı)
rfm["Monetary"] = (
    rfm["Monetary"]
    .astype(str)
    .str.replace(".", "", regex=False)
    .str.replace(",", ".", regex=False)
)
rfm["Monetary"] = pd.to_numeric(rfm["Monetary"], errors="coerce")

# 2) Frequency/Recency numeric hale getirildi
rfm["Frequency"] = pd.to_numeric(rfm["Frequency"], errors="coerce")
rfm["Recency"] = pd.to_numeric(rfm["Recency"], errors="coerce")

# 3) NaN temizle
rfm2 = rfm.dropna(subset=["Recency", "Frequency", "Monetary"]).copy()
print("rfm2 shape:", rfm2.shape)

rfm2 shape: (99996, 9)


In [23]:
import pandas as pd

snapshot_date = orders2["order_date"].max() + pd.Timedelta(days=1)

rfm = orders2.groupby("user_id").agg(
    Recency=("order_date", lambda x: (snapshot_date - x.max()).days),
    Frequency=("order_id", "nunique"),
    Monetary=("total_revenue", "sum")
).reset_index()

print("rfm shape:", rfm.shape)
print("Monetary NaN:", rfm["Monetary"].isna().sum())

rfm.head()


rfm shape: (99996, 4)
Monetary NaN: 0


,user_id,Recency,Frequency,Monetary
0,1,2,113,1132620.00
1,2,24,112,1121866.99
2,3,6,86,912249.56
3,4,6,98,917517.91
4,5,22,102,938302.75


In [24]:
import pandas as pd

rfm2 = rfm.copy()

# skorlar
rfm2["R_score"] = pd.qcut(rfm2["Recency"], q=5, labels=[5,4,3,2,1], duplicates="drop")
rfm2["F_score"] = pd.qcut(rfm2["Frequency"].rank(method="first"), q=5, labels=[1,2,3,4,5], duplicates="drop")
rfm2["M_score"] = pd.qcut(rfm2["Monetary"].rank(method="first"), q=5, labels=[1,2,3,4,5], duplicates="drop")

rfm2["R_score"] = rfm2["R_score"].astype(int)
rfm2["F_score"] = rfm2["F_score"].astype(int)
rfm2["M_score"] = rfm2["M_score"].astype(int)

rfm2["RFM_score"] = rfm2["R_score"] + rfm2["F_score"] + rfm2["M_score"]

# segment
rfm2["segment"] = "Regular"
rfm2.loc[rfm2["RFM_score"] >= 13, "segment"] = "VIP"
rfm2.loc[(rfm2["R_score"] <= 2) & (rfm2["F_score"] <= 2) & (rfm2["M_score"] <= 2), "segment"] = "At Risk"
rfm2.loc[(rfm2["R_score"] >= 4) & (rfm2["F_score"] >= 4), "segment"] = "Loyal"

rfm2["segment"].value_counts()

,count
segment,
Regular,65812
Loyal,17992
At Risk,12978
VIP,3214


In [25]:
rfm2[["Recency","Frequency","Monetary","R_score","F_score","M_score","RFM_score"]].head()

,Recency,Frequency,Monetary,R_score,F_score,M_score,RFM_score
0,2,113,1132620.00,5,5,5,15
1,24,112,1121866.99,1,5,5,11
2,6,86,912249.56,3,1,2,6
3,6,98,917517.91,3,2,2,7
4,22,102,938302.75,1,3,3,7


In [26]:
segment_summary = rfm2.groupby("segment").agg(
    customers=("user_id", "nunique"),
    avg_recency=("Recency", "mean"),
    avg_frequency=("Frequency", "mean"),
    avg_monetary=("Monetary", "mean"),
    total_revenue=("Monetary", "sum")
).reset_index().sort_values("total_revenue", ascending=False)

segment_summary

,segment,customers,avg_recency,avg_frequency,avg_monetary,total_revenue
2,Regular,65812,9.918085,101.125099,9.464624e+05,6.228859e+10
1,Loyal,17992,2.768175,112.332592,1.049267e+06,1.887840e+10
0,At Risk,12978,19.777624,91.362537,8.335321e+05,1.081758e+10
3,VIP,3214,6.651213,116.103609,1.126965e+06,3.622066e+09


## BigQuery e RFM segmentasyonunun tablo olarak aktarılması


In [27]:
rfm2[["user_id", "segment"]].sort_values("user_id")

,user_id,segment
0,1,Loyal
1,2,Regular
2,3,Regular
3,4,Regular
4,5,Regular
...,...,...
99991,99996,Regular
99992,99997,Regular
99993,99998,Regular
99994,99999,Regular


In [28]:
df_to_push = rfm2

In [31]:
project_id = "superstore-484710"
dataset_id = "dbt_superstore_int_superstore"        # VAR OLAN dataset
table_id   = "rfm_segmentation2" # YENİ veya overwrite

table_full_id = f"{project_id}.{dataset_id}.{table_id}"

In [32]:
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # tabloyu overwrite eder
    autodetect=True                      # schema otomatik
)

In [33]:
job = client.load_table_from_dataframe(
    df_to_push,
    table_full_id,
    job_config=job_config
)

job.result()  # job bitene kadar bekler
print("✅ Data successfully pushed to BigQuery:", table_full_id)

✅ Data successfully pushed to BigQuery: superstore-484710.dbt_superstore_int_superstore.rfm_segmentation2
